In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [45]:
dataset = pd.read_csv("obesity.csv")
print("Classes in the dataset:", dataset['NObeyesdad'].unique())

Classes in the dataset: ['Normal_Weight' 'Overweight_Level_I' 'Overweight_Level_II'
 'Obesity_Type_I' 'Insufficient_Weight' 'Obesity_Type_II'
 'Obesity_Type_III']


One hot encoding + Scaling the Data

In [46]:
categories = ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'CAEC', 'CALC', 'MTRANS', 'NObeyesdad']
for c in categories:
  dummies = pd.get_dummies(dataset[c], prefix=(str(c)+"_"))
  dataset = pd.concat([dataset, dummies], axis=1)
dataset = dataset.drop(columns=categories)

In [47]:
d = ['NObeyesdad__Insufficient_Weight',
       'NObeyesdad__Normal_Weight', 'NObeyesdad__Obesity_Type_I',
       'NObeyesdad__Obesity_Type_II', 'NObeyesdad__Obesity_Type_III',
       'NObeyesdad__Overweight_Level_I', 'NObeyesdad__Overweight_Level_II']
X = dataset.drop(columns=d)
y = dataset[d]

In [48]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_rescaled = scaler.fit_transform(X)
X = pd.DataFrame(data = X_rescaled, columns = X.columns)

Split the data and run the model

In [69]:
data_train, data_test, class_train, class_test = train_test_split(X, y, test_size=0.2, random_state=15)
print(data_train.shape)
print(class_train.shape)
mlp = MLPClassifier(solver = 'sgd', activation = 'logistic', learning_rate_init = 0.3, batch_size = 100, hidden_layer_sizes = (17, 20), max_iter = 500, random_state=42)

(1688, 31)
(1688, 7)


In [70]:
mlp.fit(data_train, class_train)

MLPClassifier(activation='logistic', batch_size=100,
              hidden_layer_sizes=(17, 20), learning_rate_init=0.3, max_iter=500,
              random_state=42, solver='sgd')

In [71]:
pred = mlp.predict(data_test)

In [72]:
print("Testing Accuracy : ", accuracy_score(class_test, pred))
print("Testing Mean Square Error : ", mean_squared_error(class_test, pred))

Testing Accuracy :  0.9385342789598109
Testing Mean Square Error :  0.016548463356973995


In [73]:
train_pred = mlp.predict(data_train)
print("Training Accuracy : ",accuracy_score(train_pred, class_train))
print("Training Mean Square Error : ", mean_squared_error(class_train, train_pred))

Training Accuracy :  0.9994075829383886
Training Mean Square Error :  0.00016926201760324985


The testing accuracy is around the training accuracy and the difference is around 0.06 which is sizeable but needs further testing to claim overfitting. The testing MSE is also only 0.01 larger than the training MSE, so this model is good in that regard. From what we have so far, there may not be overfitting, but let's check using Cross Validation and regularization techniques to find an improvement.

In [74]:
print(multilabel_confusion_matrix(class_test, pred))

print("Classification Report : ")
print(classification_report(class_test, pred))

[[[373   2]
  [  1  47]]

 [[363   5]
  [  6  49]]

 [[353   3]
  [  2  65]]

 [[363   1]
  [  2  57]]

 [[364   2]
  [  0  57]]

 [[351   8]
  [  7  57]]

 [[346   4]
  [  6  67]]]
Classification Report : 
              precision    recall  f1-score   support

           0       0.96      0.98      0.97        48
           1       0.91      0.89      0.90        55
           2       0.96      0.97      0.96        67
           3       0.98      0.97      0.97        59
           4       0.97      1.00      0.98        57
           5       0.88      0.89      0.88        64
           6       0.94      0.92      0.93        73

   micro avg       0.94      0.94      0.94       423
   macro avg       0.94      0.94      0.94       423
weighted avg       0.94      0.94      0.94       423
 samples avg       0.94      0.94      0.94       423



Cross Validation

In [75]:
from sklearn.model_selection import cross_validate
mlpCV = cross_validate(mlp, X, y, cv=10, scoring=['accuracy', 'neg_mean_squared_error'])
#print all the accuracy values from each iteration
print('Accuracy')
print(mlpCV['test_accuracy'])
#print all the MSE values from each iteration
print('MSE')
print(-1*mlpCV['test_neg_mean_squared_error'])
print('Average Accuracy = ',  sum(mlpCV['test_accuracy'])/len(mlpCV['test_accuracy']))
print('Average MSE = ', sum(-1 * mlpCV['test_neg_mean_squared_error']) / len(mlpCV['test_neg_mean_squared_error']))

Accuracy
[0.72641509 0.78672986 0.96208531 0.98104265 0.97630332 0.86729858
 0.95260664 0.99052133 0.87677725 1.        ]
MSE
[0.07075472 0.05416385 0.01083277 0.00338524 0.00677048 0.03317536
 0.01150982 0.00270819 0.02979012 0.        ]
Average Accuracy =  0.9119780023249575
Average MSE =  0.0223090532824057


For some folds the accuracy is lower at 72.6% and 78.6% so there is still room to improve this model, but since it performs well in 8 of the 10 folds, it is still a valuable model.


Hyperparameter Tuning

In [76]:
#MLP hyperparameter tuning

#logic from hw3demo
#set up parameters
max_iterations = [500,400,600]
hidden_layer_size = [(17, 20), (17,20,15), (17, 20, 20)]
activations = ["logistic", "relu", "tanh"]
learning_rate_inits = [0.3,0.4,0.5]
params = dict(activation = activations, hidden_layer_sizes = hidden_layer_size, max_iter = max_iterations, learning_rate_init = learning_rate_inits)
grid = GridSearchCV(estimator = mlp, param_grid=params, scoring="accuracy")
grid.fit(X, y)

GridSearchCV(estimator=MLPClassifier(activation='logistic', batch_size=100,
                                     hidden_layer_sizes=(17, 20),
                                     learning_rate_init=0.3, max_iter=500,
                                     random_state=42, solver='sgd'),
             param_grid={'activation': ['logistic', 'relu', 'tanh'],
                         'hidden_layer_sizes': [(17, 20), (17, 20, 15),
                                                (17, 20, 20)],
                         'learning_rate_init': [0.3, 0.4, 0.5],
                         'max_iter': [500, 400, 600]},
             scoring='accuracy')

In [77]:
#results
print("Optimal Hyper-Parameters:", grid.best_params_)
print("Optimal Accuracy:", grid.best_score_)

Optimal Hyper-Parameters: {'activation': 'logistic', 'hidden_layer_sizes': (17, 20), 'learning_rate_init': 0.4, 'max_iter': 500}
Optimal Accuracy: 0.8267296337378014


In [78]:
#make optimal MLP
optimalMLP = MLPClassifier(solver = 'sgd', activation = 'logistic', learning_rate_init = 0.4, batch_size = 100, hidden_layer_sizes = (17, 20), max_iter = 500, random_state=42)
#fit and predict
optimalMLP.fit(data_train, class_train)
pred = optimalMLP.predict(data_test)
#results
print("Accuracy : ", accuracy_score(class_test, pred))
print("Mean Square Error : ", mean_squared_error(class_test, pred))

Accuracy :  0.9385342789598109
Mean Square Error :  0.015535292131036813


In [79]:
#MORE results
print(multilabel_confusion_matrix(class_test, pred))

print("Classification Report : ")
print(classification_report(class_test, pred))

[[[375   0]
  [  3  45]]

 [[363   5]
  [  4  51]]

 [[354   2]
  [  1  66]]

 [[364   0]
  [  2  57]]

 [[366   0]
  [  0  57]]

 [[350   9]
  [  7  57]]

 [[343   7]
  [  6  67]]]
Classification Report : 
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        48
           1       0.91      0.93      0.92        55
           2       0.97      0.99      0.98        67
           3       1.00      0.97      0.98        59
           4       1.00      1.00      1.00        57
           5       0.86      0.89      0.88        64
           6       0.91      0.92      0.91        73

   micro avg       0.95      0.95      0.95       423
   macro avg       0.95      0.95      0.95       423
weighted avg       0.95      0.95      0.95       423
 samples avg       0.94      0.95      0.94       423



In [80]:
#Cross Validation for Optimized MLP
optimal_CV = cross_validate(optimalMLP, X, y, cv=10, scoring=['accuracy', 'neg_mean_squared_error'])
#print all the accuracy values from each iteration
print('Accuracy')
print(optimal_CV['test_accuracy'])
#print all the MSE values from each iteration
print('MSE')
print(-1*optimal_CV['test_neg_mean_squared_error'])

print('Average Accuracy = ', sum(optimal_CV['test_accuracy']) / len(optimal_CV['test_accuracy']))
print('Average MSE = ', sum(-1 * optimal_CV['test_neg_mean_squared_error']) / len(optimal_CV['test_neg_mean_squared_error']))

Accuracy
[0.66037736 0.72985782 0.94312796 0.97630332 0.8436019  0.99052133
 0.96682464 0.99052133 1.         1.        ]
MSE
[0.0902965  0.07109005 0.01557211 0.00473934 0.03452945 0.00203114
 0.00880162 0.00203114 0.         0.        ]
Average Accuracy =  0.9101135652329428
Average MSE =  0.022909135039153816


After tuning, the new model gives out a good accuracy at 93.8% but since the cross-validation for 2 of the folds are very low, we should look into different types of models like a DNN.